<a href="https://colab.research.google.com/github/doliver-app/doliver-app/blob/doliver-app-colab-ml/llm_driven_eda_vertex_langchain_palm_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# LLM-driven EDA with LangChain, Vertex and PaLM API

In [1]:
# install vertex ai palm package
! pip3 install --upgrade --quiet google-cloud-aiplatform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 15.7 MB/s eta 0:00:00


In [ ]:
# install langchain framework
!pip install langchain google-cloud-aiplatform

In [ ]:
# install langchain-experimental
!pip install langchain-experimental

In [43]:
# automatically restart kernel after installation to use new packages
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [ ]:
# authN to Google Cloud account
from google.colab import auth
auth.authenticate_user()

In [23]:
# import libraries
import pandas as pd
from langchain.llms import VertexAI
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent


In [3]:
# set project & region for Google Cloud
from google.cloud import aiplatform

aiplatform.init (

    project = 'dmo-test-access',
    location ='us-central1',
)

In [9]:
# import dataset
url = 'https://raw.githubusercontent.com/doliver-app/doliver-app/doliver-app-patch-1/SAheart.data'
df = pd.read_csv(url)

In [15]:
# define LLM objects
llm = VertexAI()
agent = create_pandas_dataframe_agent(llm, df, verbose=True)

In [17]:
# extract metadata from the dataset

In [16]:
results = agent("How many records are there?")



> Entering new AgentExecutor chain...
 Thought: The number of records is the length of the dataframe
Action: python_repl_ast
Action Input: len(df)
Observation: 462
Thought: The number of records is 462
Final Answer: 462

> Finished chain.


In [18]:
# validate LLM answer against results from code
df.shape

(462, 11)

In [20]:
agent("How many rows and columns are there?")



> Entering new AgentExecutor chain...
 Thought: The shape attribute of a dataframe contains the number of rows and columns
Action: python_repl_ast
Action Input: df.shape
Observation: (462, 11)
Thought: There are 462 rows and 11 columns
Final Answer: 462 rows and 11 columns

> Finished chain.


{'input': 'How many rows and columns are there?',
 'output': '462 rows and 11 columns'}

In [21]:
# sample a few records in dataset
results = agent('Sample 5 records and display the results')



> Entering new AgentExecutor chain...
 Thought: Use the head() function to display the first 5 records of the dataframe.
Action: python_repl_ast
Action Input: df.head()
Observation:    row.names  sbp  tobacco   ldl  adiposity  famhist  typea  obesity  alcohol  \
0          1  160    12.00  5.73      23.11  Present     49    25.30    97.20   
1          2  144     0.01  4.41      28.61   Absent     55    28.87     2.06   
2          3  118     0.08  3.48      32.28  Present     52    29.14     3.81   
3          4  170     7.50  6.41      38.03  Present     51    31.99    24.26   
4          5  134    13.60  3.50      27.78  Present     60    25.99    57.34   

   age  chd  
0   52    1  
1   63    1  
2   46    0  
3   58    1  
4   49    1  
Thought: The first 5 records of the dataframe are displayed.
Final Answer: None

> Finished chain.


In [22]:
# discover any missing values
results = agent("Are there any missing values in the dataset?")



> Entering new AgentExecutor chain...
 Thought: I can check for missing values using the `isnull()` function.
Action: python_repl_ast
Action Input: df.isnull().values.any()
Observation: False
Thought: There are no missing values in the dataset.
Final Answer: False

> Finished chain.


In [24]:
# explore frequencies of variables in dataset
results = agent("What are the unique values in the column chd?")



> Entering new AgentExecutor chain...
 Thought: The unique values in a column can be found using the unique() function.
Action: python_repl_ast
Action Input: df['chd'].unique()
Observation: [1 0]
Thought: The unique values in the column chd are 1 and 0.
Final Answer: [0, 1]

> Finished chain.


In [25]:
results = agent("What are the frequencies of the unique values in the column chd?")



> Entering new AgentExecutor chain...
 Thought: The unique values in the column chd can be found using the value_counts() method.
Action: python_repl_ast
Action Input: df['chd'].value_counts()
Observation: 0    302
1    160
Name: chd, dtype: int64
Thought: The frequencies of the unique values in the column chd are 302 and 160.
Final Answer: 0    302
1    160
Name: chd, dtype: int64

> Finished chain.


In [26]:
# validate LLM answer against results from code
df.chd.value_counts()

0    302
1    160
Name: chd, dtype: int64

In [27]:
results = agent("""If the value in the chd column in 0, then it indicates that the person is not suffering from chd. Then, how many persons
are suffering from chd?""")



> Entering new AgentExecutor chain...
 Thought: Get the count of rows where the value in the chd column is 1
Action: python_repl_ast
Action Input: df.chd.value_counts()[1]
Observation: 160
Thought: Final Answer: 160 persons are suffering from chd

> Finished chain.


In [28]:
# find probability distribution of the dataset
results = agent("What is the probability distribution of persons suffering from chd?")



> Entering new AgentExecutor chain...
 Thought: The probability distribution of persons suffering from chd can be found by using the value_counts() function on the chd column.
Action: python_repl_ast
Action Input: df['chd'].value_counts()
Observation: 0    302
1    160
Name: chd, dtype: int64
Thought: The probability distribution of persons suffering from chd is: 0: 0.654867, 1: 0.345133
Final Answer: 0: 0.654867, 1: 0.345133

> Finished chain.


In [29]:
# validate LLM answer against results from code
df.chd.value_counts(normalize = True)

0    0.65368
1    0.34632
Name: chd, dtype: float64

In [31]:
results = agent("""If the value in the chd column is 0, then it indicates that the person is not suffering from chd. Then, how many persons are
suffering from chd who did not have any family history of chd?""")



> Entering new AgentExecutor chain...
 Thought: Get all the rows where the person is suffering from chd and did not have any family history of chd
Action: python_repl_ast
Action Input: df[(df['chd'] == 1) & (df['famhist'] == 'Absent')]
Observation:      row.names  sbp  tobacco   ldl  adiposity famhist  typea  obesity  \
1            2  144     0.01  4.41      28.61  Absent     55    28.87   
10          11  206     6.00  2.95      32.27  Absent     72    26.81   
20          21  106     1.61  1.74      12.32  Absent     74    20.92   
27          28  145     9.10  5.24      27.55  Absent     59    20.96   
29          30  146     0.00  6.62      25.69  Absent     60    28.07   
..         ...  ...      ...   ...        ...     ...    ...      ...   
424        426  142     2.20  3.29      22.70  Absent     44    23.66   
446        448  142     0.00  4.32      25.22  Absent     47    28.92   
454        456  146     0.64  4.82      28.02  Absent     60    28.11   
455        457  128

In [32]:
# validate LLM answer against results from code
pd.crosstab(df.chd, df.famhist)

famhist,Absent,Present
chd,,
0,206,96
1,64,96


In [ ]:
# calculate some conditional probabilities